<a href="https://colab.research.google.com/github/patil-suraj/longbart/blob/master/LongBART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LongBART

In [1]:
!git clone https://github.com/patil-suraj/longbart.git
%cd longbart

/content/longbart


In [2]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-69qg24g6
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-69qg24g6
     |████████████████████████████████| 3.8MB 3.4MB/s 
     |████████████████████████████████| 1.1MB 59.3MB/s 
     |████████████████████████████████| 890kB 60.2MB/s 
  Created wheel for transformers: filename=transformers-2.10.0-cp36-none-any.whl size=667026 sha256=1fbfcea1f14b529238dfb962701daa0b4df4df60b6927f0793ca24f52b161af8
  Stored in directory: /tmp/pip-ephem-wheel-cache-gv3xom6x/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=87f114a7eda7007f2e152396969d392ab29f68a5812c6a57e3538111bdf7c32e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [0]:
import logging
import os
import math
from dataclasses import dataclass, field
from transformers import RobertaForMaskedLM, RobertaTokenizerFast, TextDataset, DataCollatorForLanguageModeling, Trainer
from transformers import BartTokenizer
from transformers import TrainingArguments, HfArgumentParser
from transformers.modeling_longformer import LongformerSelfAttention

from modeling_bart import BartForConditionalGeneration

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [3]:
# lets use a tiny version of bart for initial experiment 
tokenizer = BartTokenizer.from_pretrained('sshleifer/bart-tiny-random')
bart = BartForConditionalGeneration.from_pretrained('sshleifer/bart-tiny-random')

# load ROBERta model to see the difference between bart encoder layer and roberta encoder layer 
roberta = RobertaForMaskedLM.from_pretrained('roberta-base')

INFO:transformers.tokenization_utils:Model name 'sshleifer/bart-tiny-random' not found in model shortcut name list (bart-large, bart-large-mnli, bart-large-cnn, bart-large-xsum). Assuming 'sshleifer/bart-tiny-random' is a path, a model identifier, or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/sshleifer/bart-tiny-random/vocab.json from cache at /root/.cache/torch/transformers/70b9426bcc7c2cd96de53c16f7e13eabbc8373cecf5c38d68ced2fcc25e3382a.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/sshleifer/bart-tiny-random/merges.txt from cache at /root/.cache/torch/transformers/dc37af6307b1a17037d2d066cb55af9cc1cf55d38d3b1f862221fc8d87b9a672.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws

In [5]:
roberta.config

RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

In [6]:
bart.config

BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 24,
  "decoder_attention_heads": 2,
  "decoder_ffn_dim": 16,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 2,
  "decoder_max_position_embeddings": 1024,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 2,
  "encoder_ffn_dim": 16,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 2,
  "encoder_max_position_embeddings": 1024,
  "eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "model_type": "bart",
  "normalize_before": false,
  "normalize_embedding": tr

In [0]:
bart_layer = bart.model.encoder.layers[0]
roberta_layer = roberta.roberta.encoder.layer[0]

In [8]:
roberta_layer

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [9]:
bart_layer

EncoderLayer(
  (self_attn): SelfAttention(
    (k_proj): Linear(in_features=24, out_features=24, bias=True)
    (v_proj): Linear(in_features=24, out_features=24, bias=True)
    (q_proj): Linear(in_features=24, out_features=24, bias=True)
    (out_proj): Linear(in_features=24, out_features=24, bias=True)
  )
  (self_attn_layer_norm): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=24, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=24, bias=True)
  (final_layer_norm): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
)

BART calculates the output projection in the attention layer itself, also the `forward` paramter names of `SelfAttention` layer used in BART are different than that of `BertSelfAttention`. So we'll need to wrap `LongformerSelfAttention` to use it for BART

In [0]:
import math
from typing import Dict, List, Optional, Tuple

import torch
from torch import Tensor, nn

class LongformerSelfAttentionForBart(nn.Module):
  def __init__(self, config, layer_id):
    super().__init__()
    self.embed_dim = config.d_model
    self.longformer_self_attn = LongformerSelfAttention(config, layer_id=layer_id)
    self.output = nn.Linear(self.embed_dim, self.embed_dim)
  
  def forward(
    self,
    query,
    key: Optional[Tensor],
    key_padding_mask: Optional[Tensor] = None,
    layer_state: Optional[Dict[str, Optional[Tensor]]] = None,
    attn_mask: Optional[Tensor] = None,
    need_weights=False,
  ) -> Tuple[Tensor, Optional[Tensor]]:
    
    tgt_len, bsz, embed_dim = query.size()
    assert embed_dim == self.embed_dim
    assert list(query.size()) == [tgt_len, bsz, embed_dim]

    # LongformerSelfAttention expects this shape
    query = query.view(bsz, tgt_len, embed_dim)

    outputs = self.longformer_self_attn(
        query,
        attention_mask=attn_mask,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
    )

    attn_output = outputs[0] 
    attn_output = attn_output.contiguous().view(tgt_len, bsz, embed_dim)
    attn_output = self.output(attn_output)

    return (attn_output,) + outputs[1:] if len(outputs) == 2 else (attn_output, None)

In [0]:
class LongBartForConditionalGeneration(BartForConditionalGeneration):
  def __init__(self, config):
    super().__init__(config)
    for i, layer in enumerate(self.model.encoder.layers):
      # replace the `modeling_bart.SelfAttention` object with `LongformerSelfAttention`
      layer.self_attn = LongformerSelfAttentionForBart(config, layer_id=i)

In [0]:
def create_long_model(save_model_to, base_model='bart-large', attention_window=512, max_pos=4096):
    model = BartForConditionalGeneration.from_pretrained(base_model)
    tokenizer = BartTokenizer.from_pretrained('bart-large', model_max_length=max_pos)
    config = model.config

    # in BART attention_probs_dropout_prob is attention_dropout, but LongformerSelfAttention
    # expects attention_probs_dropout_prob, so set it here  
    config.attention_probs_dropout_prob = config.attention_dropout

    # extend position embeddings
    tokenizer.model_max_length = max_pos
    tokenizer.init_kwargs['model_max_length'] = max_pos
    current_max_pos, embed_size = model.model.encoder.embed_positions.weight.shape
    # config.max_position_embeddings = max_pos
    config.encoder_max_position_embeddings = max_pos
    max_pos += 2  # NOTE: BART has positions 0,1 reserved, so embedding size is max position + 2
    assert max_pos > current_max_pos
    # allocate a larger position embedding matrix
    new_pos_embed = model.model.encoder.embed_positions.weight.new_empty(max_pos, embed_size)
    # copy position embeddings over and over to initialize the new position embeddings
    k = 2
    step = current_max_pos - 2
    while k < max_pos - 1:
        new_pos_embed[k:(k + step)] = model.model.encoder.embed_positions.weight[2:]
        k += step
    model.model.encoder.embed_positions.weight.data = new_pos_embed

    # replace the `modeling_bart.SelfAttention` object with `LongformerSelfAttention`
    config.attention_window = [attention_window] * config.num_hidden_layers
    for i, layer in enumerate(model.model.encoder.layers):
        longformer_self_attn_for_bart = LongformerSelfAttentionForBart(config, layer_id=i)
        
        longformer_self_attn_for_bart.longformer_self_attn.query = layer.self_attn.q_proj
        longformer_self_attn_for_bart.longformer_self_attn.key = layer.self_attn.k_proj
        longformer_self_attn_for_bart.longformer_self_attn.value = layer.self_attn.v_proj

        longformer_self_attn_for_bart.longformer_self_attn.query_global = layer.self_attn.q_proj
        longformer_self_attn_for_bart.longformer_self_attn.key_global = layer.self_attn.k_proj
        longformer_self_attn_for_bart.longformer_self_attn.value_global = layer.self_attn.v_proj

        longformer_self_attn_for_bart.output = layer.self_attn.out_proj

        layer.self_attn = longformer_self_attn_for_bart

    logger.info(f'saving model to {save_model_to}')
    model.save_pretrained(save_model_to)
    tokenizer.save_pretrained(save_model_to)
    return model, tokenizer

In [13]:
# model_path = f'{training_args.output_dir}/roberta-base-{model_args.max_pos}'
base_model = "sshleifer/bart-tiny-random"
model_path = "bart-tiny-random-4096"
attention_window = 512
max_pos = 4096

if not os.path.exists(model_path):
    os.makedirs(model_path)

# logger.info(f'Converting roberta-base into roberta-base-{model_args.max_pos}')
model, tokenizer = create_long_model(
    save_model_to=model_path,
    base_model=base_model,
    attention_window=attention_window,
    max_pos=max_pos
)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/sshleifer/bart-tiny-random/config.json from cache at /root/.cache/torch/transformers/ce13c5b4dd7e5d8a0d2417a7842224d1535d0cd14dd928809bdb6029e1fa7af3.0a5a7d7a4a1c79b5dce5d054a64dd329deefdcbe16b8cf8a4e825bbed4186047
INFO:transformers.configuration_utils:Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 24,
  "decoder_attention_heads": 2,
  "decoder_ffn_dim": 16,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 2,
  "decoder_max_position_embeddings": 1024,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 2,
  "encoder_ffn_dim": 16,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 2,


In [14]:
long_model_tiny = LongBartForConditionalGeneration.from_pretrained('bart-tiny-random-4096')

INFO:transformers.configuration_utils:loading configuration file bart-tiny-random-4096/config.json
INFO:transformers.configuration_utils:Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "attention_window": [
    512,
    512
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 24,
  "decoder_attention_heads": 2,
  "decoder_ffn_dim": 16,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 2,
  "decoder_max_position_embeddings": 1024,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 2,
  "encoder_ffn_dim": 16,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 2,
  "encoder_max_position_embeddings": 4096,
  "eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "i

In [15]:
TXT = "My friends are <mask> but they eat too many carbs."

input_ids = tokenizer.batch_encode_plus([TXT], return_tensors='pt', max_length=4096, pad_to_max_length=True)['input_ids']

logits = long_model_tiny(input_ids)[0]

masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
probs = logits[0, masked_index].softmax(dim=0)
values, predictions = probs.topk(5)
tokenizer.decode(predictions).split()

['<unk>.<pad><s></s>']

Now lets try with bart-large

In [16]:
# model_path = f'{training_args.output_dir}/roberta-base-{model_args.max_pos}'
base_model = "bart-large"
model_path = "bart-large-4096"
attention_window = 512
max_pos = 4096

if not os.path.exists(model_path):
    os.makedirs(model_path)

# logger.info(f'Converting roberta-base into roberta-base-{model_args.max_pos}')
model, tokenizer = create_long_model(
    save_model_to=model_path,
    base_model=base_model,
    attention_window=attention_window,
    max_pos=max_pos
)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large/config.json from cache at /root/.cache/torch/transformers/7f6632e580b7d9fd4f611dd96dab877cccfc319867b53b8b72fddca7fd64de5c.40bd49bcec9d93d8b0bfbd020088e2e1b6e6bb03e8e80aa5144638f90ca6bd61
INFO:transformers.configuration_utils:Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel",
    "BartForMaskedLM",
    "BartForSequenceClassification"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_max_position_embeddings": 1024,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encod

In [7]:
long_model = LongBartForConditionalGeneration.from_pretrained('bart-large-4096')
tokenizer = BartTokenizer.from_pretrained('bart-large-4096')

INFO:transformers.configuration_utils:loading configuration file bart-large-4096/config.json
INFO:transformers.configuration_utils:Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_max_position_embeddings": 1024,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "encoder_max_position_embeddings": 4096,
  "eos_toke

In [8]:
TXT = "My friends are <mask> but they eat too many carbs."

# 4096 seq len crashes even with 35 GB memory
# so we also probably need sliding-window attention in decoder as well
input_ids = tokenizer.batch_encode_plus([TXT], return_tensors='pt', max_length=2560, pad_to_max_length=True)['input_ids']

logits = long_model(input_ids)[0]

masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
probs = logits[0, masked_index].softmax(dim=0)
values, predictions = probs.topk(5)
tokenizer.decode(predictions).split()

['having', 'still', 'going', 'getting', 'not']